In [4]:
import pykitti
import numpy as np
import open3d as o3d
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 坐标转换

In [14]:
basedir = '../datasets'
date = '2011_10_03'
drive = '0027'
dataset = pykitti.raw(basedir, date, drive)
data = dataset.get_velo(0)
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(data[:, :3])

绕坐标轴旋转

In [15]:
from scipy.spatial.transform import Rotation as R
R1 = R.from_euler('y', 90, degrees=True).as_matrix()
R2 = R.from_euler('x', -90, degrees=True).as_matrix()
o3d.visualization.draw_geometries([point_cloud.rotate(R1, center=point_cloud.get_center()).rotate(R2, center=point_cloud.get_center())])
# 复合旋转
R3 = R.from_euler('zyx', [45, 30, 60], degrees=True).as_matrix()
o3d.visualization.draw_geometries([point_cloud.rotate(R3, center=point_cloud.get_center())])

使用四元数、旋转向量旋转

In [16]:
R4 = R.from_quat([0.5, 0.5, 0.5, 0.5])
data4 = np.dot(data[:, :3], R4.as_matrix())
point_cloud4 = o3d.geometry.PointCloud()
point_cloud4.points = o3d.utility.Vector3dVector(data4)
o3d.visualization.draw_geometries([point_cloud4])

R5 = R.from_rotvec([0.1, 0.2, 0.3])
data5 = np.dot(data[:, :3], R5.as_matrix())
point_cloud5 = o3d.geometry.PointCloud()
point_cloud5.points = o3d.utility.Vector3dVector(data5)
o3d.visualization.draw_geometries([point_cloud5])

# 将GT坐标 from cam to velo

读取GT坐标、calib文件

In [5]:
def read_calib_file(filepath):
    calib_data = {}
    with open(filepath, 'r') as file:
        for line in file:
            key, value = line.split(':', 1)
            if key == 'R' or key == 'T':
                calib_data[key] = value.strip()

    return calib_data


filepath = '../datasets/2011_10_03/calib_velo_to_cam.txt'
calib_data = read_calib_file(filepath)

# 解析和转换 R 和 T 数据
if "R" in calib_data and "T" in calib_data:
    R_values = [float(x) for x in calib_data["R"].split()]
    T_values = [float(x) for x in calib_data["T"].split()]

    # 构造变换矩阵
    R_matrix = np.array(R_values).reshape(3, 3)
    T_vector = np.array(T_values).reshape(3, 1)
    RT_matrix = np.hstack((R_matrix, T_vector))
    RT_matrix = np.vstack((RT_matrix, [0, 0, 0, 1]))

    RT_matrix
else:
    RT_matrix = None

RT_matrix



array([[ 7.967514e-03, -9.999679e-01, -8.462264e-04, -1.377769e-02],
       [-2.771053e-03,  8.241710e-04, -9.999958e-01, -5.542117e-02],
       [ 9.999644e-01,  7.969825e-03, -2.764397e-03, -2.918589e-01],
       [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]])

对GT坐标进行变换

In [6]:
RT_matrix_inv = np.linalg.inv(RT_matrix)

gt_filepath = '../datasets/dataset/sequences/01/01.txt'

# 读取轨迹数据
with open(gt_filepath, 'r') as f:
    lines = f.readlines()

transformed_lines = []

for line in lines:
    pose = line.strip().split()  # 切分每行数据

     # 提取位姿变换矩阵并变为4x4
    pose_matrix = np.eye(4)
    
    pose_matrix[:3, :] = np.array(pose).reshape(3, 4).astype(float)

    # 旋转矩阵
    transformed_matrix = RT_matrix_inv @ pose_matrix @ RT_matrix

    # 从变换矩阵中提取变换后的位姿
    transformed_pose = transformed_matrix[:3, :].flatten()
    transformed_lines.append(" ".join(map(str, transformed_pose)))

# # 保存变换后的轨迹
# save_filepath = '../datasets/2011_10_03/2011_10_03_drive_0027_sync/groundtruth.txt'

# with open(save_filepath, 'w') as f:
#     f.write("\n".join(transformed_lines))

gt_velo = transformed_lines


# GroundTruth cam to velo -> velo to velo

In [7]:
data_matrices = []  # 用于存储所有的4x4矩阵
for line in gt_velo:
    pose = line.strip().split()  # 切分每行数据
    data_matrix = np.eye(4)
    data_matrix[:3, :] = np.array(pose).reshape(3, 4).astype(float)
    data_matrices.append(data_matrix)

data_matrices = np.array(data_matrices)

print(data_matrices.shape)
# print(data_matrices[5])
print(data_matrices[1000])


(1101, 4, 4)
[[-8.02950893e-02 -9.96125830e-01 -3.58592215e-02 -1.12081194e+03]
 [ 9.95078295e-01 -8.22026129e-02  5.53340574e-02 -1.74475056e+03]
 [-5.80674270e-02 -3.12396887e-02  9.97823742e-01 -6.32368301e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [12]:
import glob
import copy
# o3d可视化点云配准结果
# source 变换到 target
def draw_registration_result(source, target):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    # source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
                                      

def draw_transformed(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
                                      

def get_pointcloud(dataset, x, y):
    
    #y帧相对x帧的，x*T = y
    T = np.dot(np.linalg.inv(data_matrices[y]), data_matrices[x])
    print("T=", T)
    
    #取第x帧和第y帧的点云
    data1 = np.fromfile(dataset[x], dtype=np.float32)
    data2 = np.fromfile(dataset[y], dtype=np.float32)

    point_cloud1 = o3d.geometry.PointCloud()
    data1 = data1.reshape((-1, 4))
    # data3 = np.dot(data1, T)
    data1 = data1[:, :3]
    point_cloud1.points = o3d.utility.Vector3dVector(data1)
    print("data1=", data1)

    point_cloud2 = o3d.geometry.PointCloud()
    data2 = data2.reshape((-1, 4))
    data2 = data2[:, :3]
    point_cloud2.points = o3d.utility.Vector3dVector(data2)

    data3_pre = np.hstack((data1, np.ones((data1.shape[0], 1))))
    # print("data3_1=", data3_pre)
    data3_pre = np.dot(T, data3_pre.T).T   #A@B
    # print("data3_2=", data3_pre)
    data3 = data3_pre[:, :3] / data3_pre[:, 3].reshape((-1, 1))
    # data3 = data3[:, :3]
    print(data3.shape)
    print("data3=", data3)
    point_cloud3 = o3d.geometry.PointCloud()
    point_cloud3.points = o3d.utility.Vector3dVector(data3)

    return point_cloud1, point_cloud2, point_cloud3, T

path = '../datasets/dataset/sequences/01/velodyne'
dataset = sorted(glob.glob(os.path.join(path, '*.bin'))) 
point_cloud1, point_cloud2, point_cloud3, T = get_pointcloud(dataset, 101, 100)

#变换矩阵
draw_registration_result(point_cloud1, point_cloud2)
draw_registration_result(point_cloud3, point_cloud2)
draw_transformed(point_cloud1, point_cloud2, T)
draw_transformed(point_cloud2, point_cloud1, T)

T= [[ 9.99985620e-01  5.33548872e-03 -4.67711300e-04  2.06886576e+00]
 [-5.33487178e-03  9.99985060e-01  1.20149023e-03  1.62797477e-02]
 [ 4.74114036e-04 -1.19897595e-03  9.99999170e-01  1.99670677e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
data1= [[67.392395   10.18667     2.51537   ]
 [67.43925    10.411859    2.5173624 ]
 [22.748766    3.6098666   0.98960763]
 ...
 [ 3.9233012  -1.4436538  -1.7772034 ]
 [ 3.93826    -1.4366541  -1.7832102 ]
 [ 3.9672058  -1.434659   -1.7962162 ]]
(124823, 3)
data3= [[69.51346609  9.84629027  2.55507298]
 [69.56151808 10.07122761  2.55681766]
 [24.83610214  3.50591969  1.01603124]
 ...
 [ 5.98523919 -1.45041811 -1.7536439 ]
 [ 6.00023799 -1.4435055  -1.75965191]
 [ 6.02919998 -1.44168049 -1.77264666]]


In [24]:
# 获取真值转换的所有T
T_all = []
print(data_matrices.shape[0])
for i in range(data_matrices.shape[0] - 1):
    T = np.dot(np.linalg.inv(data_matrices[i+1]), data_matrices[i])
    T_all.append(T)

T_all = np.array(T_all)

print(T_all.shape)
print(T_all[25])

4541
(4540, 4, 4)
[[ 9.99992348e-01  5.13960176e-05 -3.91951113e-03 -9.36013499e-01]
 [-5.75011224e-05  9.99998798e-01 -1.55737962e-03 -2.08255296e-02]
 [ 3.91942455e-03  1.55759245e-03  9.99991161e-01 -1.61882689e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
